In [1]:
%run ../algorithmeX.ipynb

## [Polycubes](https://fr.wikipedia.org/wiki/Polycube)

In [17]:
import numpy as np

def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a
    
class PIECE3D:

    def __init__(self,nom,piece,couleur = None):
        self.nom = nom
        self.piece = piece
        self.couleur = couleur
        self.largeur = len(self.piece[0][0])
        self.profondeur = len(self.piece[0])
        self.hauteur = len(self.piece)

        self._g = lambda i: lambda c: np.rot90(c,i,(1,2))
        self._h = lambda c: np.rot90(np.rot90(c,1,(0,1)),2,(0,2))
        self._L = [(0,0,0),              (0,1,0), (0,1,1),             (0,2,1),     (0,3,0), (0,3,1),
                   (1,0,0), (1,0,1),     (1,1,0), (1,1,1),    (1,2,0), (1,2,1),     (1,3,0), (1,3,1),
                            (2,0,1),     (2,1,0), (2,1,1),    (2,2,0),              (2,3,0), (2,3,1),
                                         (3,1,0), (3,1,1),                          (3,3,0), (3,3,1),]


    def __hash__(self):
        return hash(self.piece)

    def __eq__(self,q):
        return self.piece == q.piece

    def __str__(self):
        return str((self.nom,self.piece))
 
    
    def isometriques(self):
        return list({PIECE3D(self.nom,totuple(self._g(i)(self._h(self._g(j)(self._h(self._g(k)(self.piece))))))) for i,j,k in self._L})

class PUZZLE3D:

    def __init__(self,pieces,
                      min_i,max_i,min_j,max_j,min_k,max_k,
                      conditions = None,
                      strict = True):
        """pieces : liste de PIECE3D
        min_i,max_i,min_j,max_j,min_k,max_k : definition du volume à remplir
        conditions : triplet d'entiers -> booleen restreignant le volume
    
        1er cas  : strict = True
            Pour chacun des noms des pieces, il faut placer 
            une et une seule piece portant ce nom.
            Les elements de E sont les noms des pieces
            et les cases (triplets d'entiers) du volume.
            Chaque element de F contient un et un seul nom
            et les cases du plateau utilisees par une piece de ce nom..
        
        2eme cas : strict = False
            Pour chaque piece de pieces, on dispose,
            pour resoudre le puzzle, d'autant d'exemplaires
            que l'on veut de la piece.
            Les elements de E sont les cases du volume.
        """
        self.min_i = min_i
        self.max_i = max_i
        self.min_j = min_j
        self.max_j = max_j
        self.min_k = min_k
        self.max_k = max_k
        self.strict = strict
        self.pieces = pieces
        if not conditions:
            conditions = lambda i,j,k: True
        lignes = dict()
        nbLignes = 0
        for p in pieces:
            for u in range(min_i, max_i - p.largeur + 2):
                for v in range(min_j, max_j - p.profondeur + 2):
                    for w in range(min_k, max_k - p.hauteur + 2):
                        ligne = [p.nom] if self.strict else []
                        match = True
                        for i in range(p.largeur):
                            if match:
                                ic = i + u 
                                for j in range(p.profondeur):
                                    if match:
                                        jc = j + v
                                        for k in range(p.hauteur):
                                            kc = k + w
                                            if p.piece[k][j][i] == 1:
                                                if conditions(ic,jc,kc):
                                                    ligne.append((ic,jc,kc))
                                                else:
                                                    match = False
                                                    break
                            else:
                                break  
                        if match:
                            lignes[nbLignes] = ligne
                            nbLignes += 1
        self.lignes = lignes
        
    def solve(self):

        F = self.lignes
        return AlgorithmeX(F).solve()

    def printSolution(self,sol):     
        for l in sol: print(self.lignes[l])

    def plotSolution(self,sol,ecart=0,**kwargs):
        """Pour afficher plusieurs solutions :
        executer un script ou utiliser jupyter
        """

        if not self.strict: 
            from random import random
            def r():
                return .6 + .4 * random()
        G = Graphics()
        for l in sol:
            e = set()
            for c in self.lignes[l]:
                if type(c) == tuple:
                    e.add(c)
                else:
                    couleur = eval(c).couleur 
            if not self.strict: couleur = (r(),r(),r())  
            couche = min([k for i,j,k in e])          
            for i,j,k in e:
                G += Polyhedron(vertices = [(i+di,j+dj,k+dk+couche*ecart) for di in [0,1] for dj in [0,1] for dk in [0,1] ]).plot(color = couleur, **kwargs)
   
        G.show(frame = False)


#------------------------------- POLYCUBES ----------------------------

Bent = PIECE3D( 'Bent',(((1,0),
                         (1,1)),), couleur = 'skyblue')
Ell = PIECE3D('Ell',(((1,0),
                      (1,0),
                      (1,1)),), couleur = 'orange')
Tee = PIECE3D('Tee',(((0,1,0),
                      (1,1,1)),), couleur = 'plum')
Skew = PIECE3D('Skew',(((0,1,1),
                        (1,1,0)),), couleur = 'darkturquoise')
Ltwist = PIECE3D('Ltwist',(((1,0),
                            (1,1)),
                           ((0,0),
                            (0,1))), couleur = 'lightpink')
Rtwist = PIECE3D('Rtwist',(((1,0),
                            (1,1)),
                           ((1,0),
                          (0,0))), couleur = 'beige')
Claw = PIECE3D('Claw',(((1,0),
                        (1,1)),
                       ((0,0),
                        (1,0))), couleur = 'cornflowerblue')

In [18]:
def somas(*args):
    puzzle = PUZZLE3D(Ell.isometriques() + Bent.isometriques() + Tee.isometriques() + Skew.isometriques() + \
                            Ltwist.isometriques() + Rtwist.isometriques() + Claw.isometriques(),*args)
    sols = puzzle.solve()
    s = next(sols)
    puzzle.plotSolution(s)

somas(1,3,1,3,1,3)
somas(1,4,1,3,1,3,lambda i,j,k: (i,j) not in [(1,1),(1,3),(4,2)])
somas(1,5,1,3,1,2,lambda i,j,k:(i,j,k) not in [(1,2,2),(3,2,2),(5,2,2)])

Graphics3d Object

Graphics3d Object

Graphics3d Object

In [19]:
def unePiece(piece,ecart,p,q,r):
    puzzle = PUZZLE3D(piece.isometriques(),1,p,1,q,1,r,strict = False)    
    sol = puzzle.solve()
    s = next(sol)
    puzzle.plotSolution(s, ecart = ecart)

unePiece(Ltwist,2,4,4,4)

Graphics3d Object

In [20]:
unePiece(Claw,0,4,4,2)

Graphics3d Object

In [21]:
unePiece(Skew,3,4,4,4)

Graphics3d Object

In [22]:
unePiece(Bent,2,3,3,3)

Graphics3d Object

In [23]:
# 2528 solutions essentielles

Arthur = PIECE3D('Arthur',(((0,1,0,0),
                            (1,1,1,1)),))

unePiece(Arthur,4,5,5,5)

Graphics3d Object